In [6]:
import os
import numpy as np
import pandas as pd
import audiofile
import opensmile
from pathlib import Path
from scipy.stats import ttest_ind, mannwhitneyu, shapiro
from statistics import mean
from tqdm import tqdm

# Initialize OpenSMILE feature extractor
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals,
)
features_names = smile.feature_names

# Selected features for analysis
SELECTED_FEATURES = [
    'F0semitoneFrom27.5Hz_sma3nz_amean', 'F0semitoneFrom27.5Hz_sma3nz_stddevNorm',
    'loudness_sma3_amean', 'loudness_sma3_stddevNorm', 
    'spectralFlux_sma3_amean', 'spectralFlux_sma3_stddevNorm', 
    'mfcc1_sma3_amean', 'mfcc1_sma3_stddevNorm', 
    'mfcc2_sma3_amean', 'mfcc2_sma3_stddevNorm',
    'mfcc3_sma3_amean', 'mfcc3_sma3_stddevNorm', 
    'jitterLocal_sma3nz_amean', 'jitterLocal_sma3nz_stddevNorm', 
    'shimmerLocaldB_sma3nz_amean', 'shimmerLocaldB_sma3nz_stddevNorm',
    'F1frequency_sma3nz_amean', 'F1frequency_sma3nz_stddevNorm', 
    'F1bandwidth_sma3nz_amean', 'F1bandwidth_sma3nz_stddevNorm', 
    'F2frequency_sma3nz_amean', 'F2frequency_sma3nz_stddevNorm', 
    'F2bandwidth_sma3nz_amean', 'F2bandwidth_sma3nz_stddevNorm', 
    'F3frequency_sma3nz_amean', 'F3frequency_sma3nz_stddevNorm', 
    'F3bandwidth_sma3nz_amean', 'F3bandwidth_sma3nz_stddevNorm'
]

def extract_features(file_path):
    """Extract audio features using OpenSMILE"""
    signal, sampling_rate = audiofile.read(file_path, always_2d=True)
    result = smile.process_signal(signal, sampling_rate)
    return list(result.iloc[0])

def process_audio_files(directory_path):
    """Process all audio files in directory and return features DataFrame"""
    features = []
    file_names = []
    
    for file_path in tqdm(list(directory_path.iterdir()), desc="Processing audio files"):
        if file_path.suffix.lower() in ['.wav', '.flac']:
            features.append(extract_features(str(file_path)))
            file_names.append(file_path.name)
    
    df = pd.DataFrame(features, columns=features_names, dtype=float)
    df['file_name'] = file_names
    df['group'] = df['file_name'].apply(lambda x: 'control' if 'hc' in x.lower() else 'patient')
    return df

def check_distribution_normality(df, group_name):
    """Check feature distribution normality using Shapiro-Wilk test"""
    results = []
    print(f"\nChecking distribution normality for {group_name} group:")
    
    for column in SELECTED_FEATURES:
        stat, p_value = shapiro(df[column])
        is_normal = p_value >= 0.05
        results.append({
            'feature': column,
            'statistic': stat,
            'p_value': p_value,
            'distribution': 'normal' if is_normal else 'non-normal'
        })
        print(f"{column}: {'Normal' if is_normal else 'Non-normal'} distribution (p={p_value:.4f})")
    
    return pd.DataFrame(results)

def compare_groups(control_df, patient_df, normality_results):
    """Compare groups using appropriate statistical tests"""
    comparison_results = []
    
    for feature in SELECTED_FEATURES:
        # Determine which test to use based on distribution normality
        control_normal = normality_results.loc[
            (normality_results['feature'] == feature) & 
            (normality_results['group'] == 'control'), 
            'distribution'
        ].values[0] == 'normal'
        
        patient_normal = normality_results.loc[
            (normality_results['feature'] == feature) & 
            (normality_results['group'] == 'patient'), 
            'distribution'
        ].values[0] == 'normal'
        
        if control_normal and patient_normal:
            # Use t-test for normally distributed data
            stat, p_value = ttest_ind(patient_df[feature], control_df[feature])
            test_used = 't-test'
        else:
            # Use Mann-Whitney U test for non-normal distributions
            stat, p_value = mannwhitneyu(patient_df[feature], control_df[feature])
            test_used = 'Mann-Whitney U'
        
        is_significant = p_value < 0.05
        comparison_results.append({
            'feature': feature,
            'test_used': test_used,
            'statistic': stat,
            'p_value': p_value,
            'significant_difference': is_significant,
            'control_mean': mean(control_df[feature]),
            'patient_mean': mean(patient_df[feature]),
            'control_min': min(control_df[feature]),
            'control_max': max(control_df[feature]),
            'patient_min': min(patient_df[feature]),
            'patient_max': max(patient_df[feature])
        })
    
    return pd.DataFrame(comparison_results)

def main():
    # Set paths and process files
    notebook_dir = Path(os.getcwd()).parent.parent 
    audio_dir = notebook_dir / "sc_new" / "data" / "all_audio_files"
    output_dir = Path("./results")
    output_dir.mkdir(exist_ok=True)
    
    # Process all audio files
    print("Extracting features from audio files...")
    all_features_df = process_audio_files(audio_dir)
    
    # Split into control and patient groups
    control_df = all_features_df[all_features_df['group'] == 'control'].copy()
    patient_df = all_features_df[all_features_df['group'] == 'patient'].copy()
    
    # Select only the features we're interested in
    control_df = control_df[SELECTED_FEATURES + ['file_name']]
    patient_df = patient_df[SELECTED_FEATURES + ['file_name']]
    
    # Save raw features
    all_features_df.to_csv(output_dir / "all_audio_features.csv", index=False)
    control_df.to_csv(output_dir / "control_group_features.csv", index=False)
    patient_df.to_csv(output_dir / "patient_group_features.csv", index=False)
    
    # Check distribution normality
    control_normality = check_distribution_normality(control_df, "control")
    patient_normality = check_distribution_normality(patient_df, "patient")
    
    # Combine normality results
    control_normality['group'] = 'control'
    patient_normality['group'] = 'patient'
    normality_results = pd.concat([control_normality, patient_normality])
    normality_results.to_csv(output_dir / "normality_test_results.csv", index=False)
    
    # Compare groups
    comparison_results = compare_groups(
        control_df[SELECTED_FEATURES], 
        patient_df[SELECTED_FEATURES], 
        normality_results
    )
    
    # Save comparison results
    comparison_results.to_csv(output_dir / "group_comparison_results.csv", index=False)
    
    # Print significant differences
    print("\nSignificant differences between groups:")
    significant_results = comparison_results[comparison_results['significant_difference']]
    if significant_results.empty:
        print("No significant differences found at p < 0.05")
    else:
        for _, row in significant_results.iterrows():
            print(f"{row['feature']}: p = {row['p_value']:.4f} ({row['test_used']})")
    
    print("Processing complete.")

if __name__ == "__main__":
    main()

Extracting features from audio files...


Processing audio files: 100%|██████████| 73/73 [09:05<00:00,  7.48s/it]


Checking distribution normality for control group:
F0semitoneFrom27.5Hz_sma3nz_amean: Normal distribution (p=0.1084)
F0semitoneFrom27.5Hz_sma3nz_stddevNorm: Non-normal distribution (p=0.0202)
loudness_sma3_amean: Non-normal distribution (p=0.0000)
loudness_sma3_stddevNorm: Non-normal distribution (p=0.0010)
spectralFlux_sma3_amean: Non-normal distribution (p=0.0000)
spectralFlux_sma3_stddevNorm: Non-normal distribution (p=0.0001)
mfcc1_sma3_amean: Normal distribution (p=0.1645)
mfcc1_sma3_stddevNorm: Normal distribution (p=0.5495)
mfcc2_sma3_amean: Normal distribution (p=0.8899)
mfcc2_sma3_stddevNorm: Non-normal distribution (p=0.0000)
mfcc3_sma3_amean: Non-normal distribution (p=0.0097)
mfcc3_sma3_stddevNorm: Non-normal distribution (p=0.0000)
jitterLocal_sma3nz_amean: Normal distribution (p=0.1131)
jitterLocal_sma3nz_stddevNorm: Normal distribution (p=0.9833)
shimmerLocaldB_sma3nz_amean: Normal distribution (p=0.3801)
shimmerLocaldB_sma3nz_stddevNorm: Normal distribution (p=0.0811)
